**Image preparation**

Firstly all libaries will be imported.

In [1]:
import cv2 as cv
import os
import numpy as np

Images of the subjects will be taken in specified categories

Define output folder for the processed and cropped images

In [2]:
output_folders = ["Processed/Hex", "Processed/Machine", "Processed/Philips", "Processed/Torx"]
cropped_folders = ["Cropped/Hex", "Cropped/Machine", "Cropped/Philips", "Cropped/Torx"]
